In [19]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../src")
import utils.munging_utils as query
import utils.preprocessing_utils as process
import pandas as npd
import numpy as np
import warnings
warnings.simplefilter("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
data = query.get_file_entity("syn21281633")
max_data = process.collapseFeatures(aggregation_type = "max").transform(data)

In [ ]:
## NULL MODEL ## 